Rokas Petrauskas 1gr. 2pogr.

Importai

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from PIL import Image
from torchvision.transforms import ToTensor, Normalize, Compose
import os
import numpy as np

gdrive mount

In [ ]:
# google drive

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
google_drive_dir = "/content/gdrive/MyDrive/gmm_lab3/data"

Pic download, creating directories and moving pictures to required directories.

Commented so it is not run when running all code since all the data is stored in google drive.

In [ ]:
# import os
# import fiftyone as fo
# import fiftyone.zoo as foz
# import random
# import shutil


# # vienos klases nuotrauku ir ju kaukiu atsiuntimas & export
# def exportClassDataset(className, num_samples = 10):
#   dataset_class = foz.load_zoo_dataset(
#     "open-images-v6",
#     "train",
#     label_types=["segmentations"],
#     classes=[className],
#     max_samples=num_samples,
#     seed=51,
#     shuffle=False,
#     dataset_name="open-images-"+className,
#     overwrite=True
#   )

#   set_dir = os.path.join(google_drive_dir, className, "all")
#   os.makedirs(set_dir, exist_ok=True)

#   dataset_class.export(
#       export_dir=set_dir,
#       dataset_type=fo.types.ImageSegmentationDirectory
#   )

#   fo.delete_dataset("open-images-"+className)

#   return set_dir

# # padalinimas i train/validation/test directories su 0.7/0.2/0.1 santykiu
# def splitExported(set_dir, className):
#   train_dir = os.path.join(google_drive_dir, className, "train")
#   validation_dir = os.path.join(google_drive_dir, className, "validation")
#   test_dir = os.path.join(google_drive_dir, className, "test")

#   os.makedirs(train_dir, exist_ok=True)
#   os.makedirs(validation_dir, exist_ok=True)
#   os.makedirs(test_dir, exist_ok=True)

#   # Set the split ratios
#   train_split = 0.7
#   validation_split = 0.2
#   test_split = 0.1

#   # Get all image filenames
#   all_images_dir = os.path.join(set_dir, "data")
#   all_masks_dir = os.path.join(set_dir, "labels")

#   all_images = os.listdir(all_images_dir)

#   # Shuffle the filenames
#   random.seed(42)
#   random.shuffle(all_images)

#   # Calculate the number of samples for each split
#   num_train = int(len(all_images) * train_split)
#   num_validation = int(len(all_images) * validation_split)

#   # Split the data
#   train_images = all_images[:num_train]
#   validation_images = all_images[num_train:num_train + num_validation]
#   test_images = all_images[num_train + num_validation:]

#   # Move the files to the respective directories
#   def move_files(src_images_dir, src_masks_dir, dst_images_dir, dst_masks_dir, filenames):
#       for filename in filenames:
#           src_image_path = os.path.join(src_images_dir, filename)
#           dst_image_path = os.path.join(dst_images_dir, filename)
#           shutil.move(src_image_path, dst_image_path)

#           mask_filename = os.path.splitext(filename)[0] + ".png"
#           src_mask_path = os.path.join(src_masks_dir, mask_filename)
#           dst_mask_path = os.path.join(dst_masks_dir, mask_filename)
#           shutil.move(src_mask_path, dst_mask_path)

#   os.makedirs(os.path.join(train_dir, "data"), exist_ok=True)
#   os.makedirs(os.path.join(train_dir, "labels"), exist_ok=True)
#   move_files(all_images_dir, all_masks_dir, os.path.join(train_dir, "data"), os.path.join(train_dir, "labels"), train_images)

#   os.makedirs(os.path.join(validation_dir, "data"), exist_ok=True)
#   os.makedirs(os.path.join(validation_dir, "labels"), exist_ok=True)
#   move_files(all_images_dir, all_masks_dir, os.path.join(validation_dir, "data"), os.path.join(validation_dir, "labels"), validation_images)

#   os.makedirs(os.path.join(test_dir, "data"), exist_ok=True)
#   os.makedirs(os.path.join(test_dir, "labels"), exist_ok=True)
#   move_files(all_images_dir, all_masks_dir, os.path.join(test_dir, "data"), os.path.join(test_dir, "labels"), test_images)

#   # Clean up empty "all" directories
#   shutil.rmtree(all_images_dir)
#   shutil.rmtree(all_masks_dir)

In [ ]:
# # nuotrauku atsisiuntimas, eksportavimas ir paskirstymas

# classes = ["Banana", "Bear", "Sock"]

# for class_i in classes:
#   print(class_i)
#   dir = exportClassDataset(class_i, 100)
#   splitExported(dir, class_i)


Moving all to GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


data loader

custom dataset is needed to load picture with the corresponding mask.

In [ ]:
data_dirs = [
        "/content/gdrive/MyDrive/gmm_lab3/data/Banana",
        "/content/gdrive/MyDrive/gmm_lab3/data/Bear",
        "/content/gdrive/MyDrive/gmm_lab3/data/Sock",
    ]

n_classes = 4
input_size = (256, 256)
batch_size = 16
n_epochs = 2

# class CustomDataset(Dataset):
#     def __init__(self, data_dirs, mode, transform=None):
#         self.data_dirs = data_dirs
#         self.mode = mode
#         self.transform = transform
#         self.image_label_list = []
        
#         for class_num, data_dir in enumerate(data_dirs, 2):
#             image_list = sorted(os.listdir(os.path.join(data_dir, mode, "data")))
#             label_list = sorted(os.listdir(os.path.join(data_dir, mode, "labels")))
            
#             for image_file, label_file in zip(image_list, label_list):
#                 image_path = os.path.join(data_dir, mode, "data", image_file)
#                 label_path = os.path.join(data_dir, mode, "labels", label_file)
#                 self.image_label_list.append((image_path, label_path, class_num))

#     def __len__(self):
#         return len(self.image_label_list)

#     def __getitem__(self, idx):
#         image_path, label_path, class_num = self.image_label_list[idx]
#         image = Image.open(image_path).convert("RGB")
#         label = Image.open(label_path).convert("L")
#         label = (np.array(label) > 0).astype(np.int32) * (class_num - 1)
        
#         if self.transform:
#             image = self.transform(image)
#             label = transforms.Resize((224, 224))(label)

#         # label = torch.from_numpy(label)
            
#         return image, label

class CustomDataset(Dataset):
    def __init__(self, data_dirs, mode, transform=None):
        self.data_dirs = data_dirs
        self.mode = mode
        self.transform = transform
        self.image_label_list = []
        
        for class_num, data_dir in enumerate(data_dirs, 2):
            image_list = sorted(os.listdir(os.path.join(data_dir, mode, "data")))
            label_list = sorted(os.listdir(os.path.join(data_dir, mode, "labels")))
            
            for image_file, label_file in zip(image_list, label_list):
                image_path = os.path.join(data_dir, mode, "data", image_file)
                label_path = os.path.join(data_dir, mode, "labels", label_file)
                self.image_label_list.append((image_path, label_path, class_num))

    def __len__(self):
        return len(self.image_label_list)

    def __getitem__(self, idx):
        image_path, label_path, class_num = self.image_label_list[idx]
        image = Image.open(image_path).convert("RGB")
        label = Image.open(label_path).convert("L")
        label = (np.array(label) > 0).astype(np.int32) * (class_num + 1)
        
        if self.transform:
            image = self.transform(image)
            label = np.array(label)
            label = Image.fromarray(label)
            label = transforms.Resize((224, 224))(label)
            label = np.array(label)

        label = torch.from_numpy(label)
            
        return image, label

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(20),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# def custom_collate(batch):
#     images = []
#     labels = []

#     for item in batch:
#         images.append(item[0])
#         labels.append(item[1])

#     images = torch.stack(images, dim=0)
#     labels = torch.stack(labels, dim=0)

#     return images, labels

train_dataset = CustomDataset(data_dirs, "train", transform = transform_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


test_dataset = CustomDataset(data_dirs, "test", transform = transform_test)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)


---

**KLASES**: BANANA, BEAR, SOCK

---



modelio architektura

In [ ]:
class SegmentationModel(nn.Module):
    def __init__(self, n_classes=4):
        super(SegmentationModel, self).__init__()
        self.n_classes = n_classes
        # Define convolutional layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        # Define final segmentation layer
        self.segmentation_layer = nn.Conv2d(256, n_classes, kernel_size=1)

    def forward(self, x):
        # Pass input through convolutional layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        # Pass through segmentation layer and apply softmax
        x = self.segmentation_layer(x)
        x = F.softmax(x, dim=1)
        return x

loss function and optimizer used a few cells lower. changed to fix bugs.

In [ ]:
# model = SegmentationModel(n_classes=4)
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train loop

In [ ]:
# def train(model, train_loader, test_loader, optimizer, criterion, n_epochs):
#     for epoch in range(n_epochs):
#         # Train
#         model.train()
#         train_loss = 0.0
#         for batch_idx, (data, target) in enumerate(train_loader):
#             optimizer.zero_grad()
#             output = model(data)
#             loss = criterion(output, target)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss /= len(train_loader)
        
#         # Evaluate on test set
#         model.eval()
#         test_loss = 0.0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for batch_idx, (data, target) in enumerate(test_loader):
#                 output = model(data)
#                 loss = criterion(output, target)
#                 test_loss += loss.item()
                
#         test_loss /= len(test_loader)
        
#         # Print progress
#         print('Epoch: {}, Train Loss: {:.4f}, Test Loss: {:.4f}'.format(epoch+1, train_loss, test_loss))

def train(model, train_loader, test_loader, optimizer, criterion, n_epochs):
    for epoch in range(n_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())  # convert labels to long data type since it throws an error with int32
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1}, loss: {running_loss/len(train_loader)}")

This defines the model, optimizer and loss function

the training loop is started here

In [ ]:
model = SegmentationModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = F.nll_loss
n_epochs = 10

# train(model, train_loader, test_loader, optimizer, criterion, n_epochs)

In [ ]:
PATH = '/content/gdrive/MyDrive/gmm_lab3/model.pth'

# # save
# torch.save(model.state_dict(), PATH)

# # reload
model = SegmentationModel()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
# print_freq = len(train_loader)

# for epoch in range(n_epochs):
#     # Set the model to train mode
#     model.train()

#     # Iterate over the batches in the train loader
#     for batch_idx, (images, labels) in enumerate(train_loader):
#         # Zero out the gradients
#         optimizer.zero_grad()

#         # Feed the batch through the model and obtain the model's predictions
#         logits = model(images)

#         # Calculate the loss between the predictions and the ground truth labels
#         loss = loss_fn(logits, labels)

#         # Backpropagate the loss to calculate gradients
#         loss.backward()

#         # Update the model's parameters using the gradients
#         optimizer.step()

#         # Print the loss and accuracy at regular intervals
#         if batch_idx % print_freq == 0:
#             print(f"Epoch [{epoch}/{n_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

# # # save
# PATH = '/content/gdrive/MyDrive/gmm_lab3/LOCAL01.pth'
# torch.save(model.state_dict(), PATH)
# # # reload
# # net = Net()
# # net.load_state_dict(torch.load(PATH))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

model.eval()

def dice_coefficient(prediction, target, smooth=1.):
    prediction = torch.from_numpy(prediction).reshape(-1)
    target = target.reshape(-1)
    intersection = (prediction * target).sum()
    dice = (2. * intersection + smooth) / (prediction.sum() + target.sum() + smooth)
    return dice.item()

with torch.no_grad():
    dice_scores = []
    micro_f1_scores = []
    macro_f1_scores = []
    test_loss = 0
    true_labels_list = []
    pred_labels_list = []
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels.to(torch.long).to(device))
        test_loss += loss.item() * images.size(0)

        predicted_labels = torch.argmax(outputs, dim=1)
        predicted_labels = predicted_labels.cpu().numpy()
        true_labels = labels.cpu().numpy()

        for i in range(len(true_labels)):
            dice_score = dice_coefficient(predicted_labels[i], true_labels[i])
            dice_scores.append(dice_score)
            
            true_labels_list.extend(list(true_labels[i].flatten()))
            pred_labels_list.extend(list(predicted_labels[i].flatten()))

        # Display a few images and predicted labels
        for i in range(3):
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(np.transpose(images[i], (1, 2, 0)))
            plt.title("Image")
            plt.axis("off")

            plt.subplot(1, 2, 2)
            plt.imshow(predicted_labels[i])
            plt.title("Predicted label")
            plt.axis("off")

            plt.show()

    average_dice = np.mean(dice_scores)
    test_loss /= len(test_loader.dataset)

    # Calculate micro- and macro-averaged F1 scores
    micro_f1 = f1_score(true_labels_list, pred_labels_list, average='micro')
    macro_f1 = f1_score(true_labels_list, pred_labels_list, average='macro')

    print(f"Test Dice coefficient: {average_dice:.4f}")
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test Micro-F1: {micro_f1:.4f}")
    print(f"Test Macro-F1: {macro_f1:.4f}")
